In [1]:
import numpy as np
import pandas as pd

In [47]:
TRAIN_DATA_FILE = 'SPAM_DATA_MINE/02-Training/train-data.txt'
TEST_DATA_FILE = 'SPAM_DATA_MINE/02-Training/test-data.txt'

TOKEN_SPAM_PROB_FILE = 'SPAM_DATA_MINE/02-Training/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SPAM_DATA_MINE/02-Training/prob-ham.txt'
TOKEN_ALL_PROB_FILE = 'SPAM_DATA_MINE/02-Training/prob-all.txt'

TEST_FEATURE_MAT = 'SPAM_DATA_MINE/02-Training/test-feature.txt'
TEST_TARGET_FILE = 'SPAM_DATA_MINE/02-Training/test-target.txt'



VOCAB_SIZE = 2500

In [3]:
sparse_train_data = np.loadtxt(TRAIN_DATA_FILE,delimiter=' ',dtype=int)

In [4]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE,delimiter=' ',dtype=int)

In [5]:
sparse_train_data[:5]

array([[ 0,  1,  2,  1],
       [ 0,  1,  3,  2],
       [ 0,  1,  4,  1],
       [ 0,  1,  7,  3],
       [ 0,  1, 11,  1]])

In [6]:
sparse_train_data[-5:]

array([[5795,    0, 2046,    2],
       [5795,    0, 2051,    1],
       [5795,    0, 2282,    1],
       [5795,    0, 2370,    1],
       [5795,    0, 2435,    1]])

In [7]:
# num of rows in traning file
sparse_train_data.shape[0]

258322

In [8]:
#num of rows in test file
sparse_test_data.shape[0]

117618

In [9]:
sparse_test_data[:5]

array([[8, 1, 2, 1],
       [8, 1, 3, 4],
       [8, 1, 4, 2],
       [8, 1, 5, 1],
       [8, 1, 6, 2]])

In [10]:
# num of emails in training file
np.unique(sparse_train_data[:,0]).size


4015

In [11]:
# num of emails in test file
np.unique(sparse_test_data[:,0]).size

1724

## Create a Empty DataFrame

In [12]:
col_name = ['DOC_ID'] + ['CATEGORY'] + list(range(0,VOCAB_SIZE))
col_name[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [13]:
len(col_name)

2502

In [14]:
type(col_name)

list

In [15]:
index_name = np.unique(sparse_train_data[:,0])
index_name

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [16]:
full_train_data = pd.DataFrame(index=index_name,columns=col_name)

In [17]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# Replace Nan Value with 0

full_train_data.fillna(value=0,inplace=True)

In [19]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
sparse_train_data[10:14]

array([[ 0,  1, 31,  1],
       [ 0,  1, 33,  1],
       [ 0,  1, 45,  1],
       [ 0,  1, 52,  1]])

## Creating full Matrix from Sparse

In [21]:
def make_full_matrix(sparse_matrix,nr_words,doc_idx=0,word_idx=2,cat_idx=1,freq_idx=3):
    
    col_name = ['DOC_ID'] + ['CATEGORY'] + list(range(0,VOCAB_SIZE))
    doc_id_name = np.unique(sparse_matrix[:,0])
    full_matrix = pd.DataFrame(index=doc_id_name,columns=col_name)
    full_matrix.fillna(value=0,inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr,'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr,'CATEGORY'] = label
        full_matrix.at[doc_nr,word_id] = occurence
        
    full_matrix.set_index('DOC_ID',inplace=True)
    return full_matrix
    

In [22]:
%%time
full_train_data = make_full_matrix(sparse_train_data,VOCAB_SIZE)

Wall time: 18 s


In [23]:
full_train_data.tail()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
5789,0,3,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,1,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,3,1,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5794,0,1,1,1,0,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
5795,0,3,4,2,0,5,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0


## Training the Model

In [24]:
full_train_data.CATEGORY.size

4015

In [25]:
# Probability of SPAM
prob_spam = full_train_data.CATEGORY.sum()/full_train_data.CATEGORY.size
prob_spam

0.31133250311332505

In [26]:
full_train_features = full_train_data.drop(['CATEGORY'],axis=1)
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,13,0,0,...,0,1,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
email_length = full_train_features.sum(axis=1)
email_length

DOC_ID
0         87
1         53
2         40
3        182
4         43
5        141
6         46
7        111
9         42
10        35
11        15
13        64
16        46
18        35
20       172
21       166
24        12
25       169
27        14
28        60
31        53
32       137
34         4
35       902
36        43
39        80
40        68
41         2
42        46
46        36
        ... 
5753      24
5754      21
5756      43
5758      50
5759      76
5760      62
5761     202
5764      86
5765      46
5766     134
5767      82
5768     161
5770      12
5771       8
5772     180
5773      76
5775    6867
5776      82
5777      42
5779      18
5781      68
5782       4
5784     820
5785     137
5787      38
5789      18
5790      72
5791      45
5794      57
5795     264
Length: 4015, dtype: int64

In [28]:
total_wc = email_length.sum()
total_wc

430919

## Num of Tokens in Spam and Ham Emails

In [29]:
spam_lengths = email_length[full_train_data.CATEGORY == 1]
spam_lengths.shape

(1250,)

In [30]:
spam_wc = spam_lengths.sum()
spam_wc

178661

In [31]:
ham_lengths = email_length[full_train_data.CATEGORY == 0]
ham_lengths.shape

(2765,)

In [32]:
nonspam_wc = ham_lengths.sum()
nonspam_wc

252258

In [33]:
spam_wc+nonspam_wc - total_wc

0

### Summing the Tokens in Spam

In [34]:
full_train_features.shape

(4015, 2500)

In [35]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,13,0,0,...,0,1,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
train_spam_tokens.tail()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1885,1,0,0,2,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1887,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1889,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1890,2,0,0,0,1,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1895,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
train_spam_tokens.shape

(1250, 2500)

In [38]:
summed_spam_token = train_spam_tokens.sum(axis=0) + 1 # Adding 1 so that we can avoid dividing 0
summed_spam_token.shape

(2500,)

In [39]:
summed_spam_token

0       2177
1        936
2       1220
3       2026
4       1211
5       1100
6        971
7       1661
8        822
9        652
10       665
11       884
12       549
13       475
14       690
15       764
16       971
17      1308
18      1285
19      1452
20      1152
21      1004
22       690
23       897
24       604
25      1079
26      1147
27       441
28       496
29       602
        ... 
2470      11
2471      20
2472      20
2473       2
2474      14
2475       1
2476      11
2477      16
2478       6
2479      23
2480      29
2481       1
2482      26
2483      28
2484       3
2485      19
2486       3
2487      30
2488      22
2489       7
2490      23
2491      28
2492       7
2493       3
2494      19
2495       1
2496      23
2497      22
2498      25
2499      12
Length: 2500, dtype: int64

In [40]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
summed_ham_token = train_ham_tokens.sum() + 1 # Adding 1 so that we can avoid dividing 0
summed_ham_token.shape

(2500,)

 ## P(Token | Spam) - Probability that a token occurs given that email is Spam

In [41]:
prob_token_spam = summed_spam_token /(spam_wc + VOCAB_SIZE)
prob_token_spam[:5]

0    0.012017
1    0.005167
2    0.006734
3    0.011183
4    0.006685
dtype: float64

In [42]:
prob_token_spam.sum()

1.0

## P(Token | Ham) - Probability that a Token Occurs given the Email is Nonspam

In [43]:
prob_token_ham = summed_ham_token/(nonspam_wc+ VOCAB_SIZE)
prob_token_ham[:5]

0    0.021522
1    0.010159
2    0.008023
3    0.003682
4    0.006324
dtype: float64

In [44]:
prob_token_ham.sum()

1.0

## P(Token) - Probability that a token will occur

In [45]:
prob_token_all = full_train_features.sum(axis=0)/ total_wc

In [46]:
prob_token_all.sum()

0.9999999999999999

## Save the trained model

In [48]:
np.savetxt(TOKEN_SPAM_PROB_FILE,prob_token_spam)
np.savetxt(TOKEN_HAM_PROB_FILE,prob_token_ham)
np.savetxt(TOKEN_ALL_PROB_FILE,prob_token_all)

## Prepare Test Data

In [49]:
sparse_test_data.shape

(117618, 4)

In [50]:
%%time
full_test_data = make_full_matrix(sparse_test_data,nr_words=VOCAB_SIZE)

Wall time: 7.64 s


In [51]:
full_test_data.columns

Index(['CATEGORY',          0,          1,          2,          3,          4,
                5,          6,          7,          8,
       ...
             2490,       2491,       2492,       2493,       2494,       2495,
             2496,       2497,       2498,       2499],
      dtype='object', length=2501)

In [52]:
X_test = full_test_data.loc[:,full_test_data.columns !='CATEGORY']
y_test = full_test_data.CATEGORY

In [53]:
np.savetxt(TEST_FEATURE_MAT,X_test)
np.savetxt(TEST_TARGET_FILE,y_test)